In [1]:
from createDataSet import createDataSet 
data = createDataSet('../data/2018_04_14/listings.csv','../data/2018_04_14/reviews.csv').create()

In [274]:
from calendar_clean_exploration import clean_calendar
import datetime,calendar
import pandas as pd

In [262]:
def holiday(year):
    #fixed holidays
    fixmd = [[1,1],[7,4],[11,11]]
    holidays = [datetime.datetime(year,x[0],x[1]) for x in fixmd]
    #floating holidays
    #'2,3,1': the third Monday of Feburary;'5,-1,1': the last Monday of May 
    floatmwd = ['1,3,1','2,3,1','5,-1,1','9,1,1','10,2,1','11,4,4']
    holidays.extend([flholiday(year,x) for x in floatmwd])
    return sorted(holidays)
    
def flholiday(y,mwd):
    #input year and month-week-weekday
    m,w,d = list(map(int,mwd.split(',')))
    d = d-1
    wday, mrange = calendar.monthrange(y, m) #firstDayWeekDay,monthRange
    if w>0:
        if wday<=d:
            day = 1+7*(w-1)+d-wday
        else:
            day = 1+7*w-(wday-d)
    else:
        wday = datetime.datetime(y,m,mrange).weekday()
        if d<=wday:
            day = mrange+7*(w+1)-(wday-d)
        else:
            day = mrange+7*w+(d-wday)
    return datetime.datetime(y,m,day)           

In [275]:
df = clean_calendar('../data/2018_04_14/calendar.csv')
#return 0-6, stands for Mon to Sun
df['week'] = df.date.dt.dayofweek

In [223]:
mergedata = pd.merge(data.drop(columns='price'),df,left_on='id',right_on='listing_id',how='inner').drop(columns = 'listing_id')

In [238]:
#day of week
week = ['Mon','Tue','Wed','Thu','Fri','Sat','Sun']
for i,w in enumerate(week):
    mergedata[w]=False
    mergedata.loc[mergedata['week']==i,w]=True

In [240]:
#which month
month = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
for i,m in enumerate(month):
    mergedata[m]=False
    mergedata.loc[mergedata.date.dt.month==i+1,m]=True

In [243]:
#which year
year = [2018,2019]
for y in year:
    mergedata[y]=False
    mergedata.loc[mergedata.date.dt.year==y,y]=True

In [257]:
#christams holidays: 12/25-12/31
mergedata['ChristmasHolidays']=False
mergedata.loc[(mergedata.date.dt.month==12) & (mergedata.date.dt.day>=25),'ChristmasHolidays']=True

In [267]:
#other holidays
holidays = ['NewYear','MartinLK','President','Memorial','Independence','Labor','Columbus','Veterans','Thanksgiving']
holiday18 = holiday(2018)
holiday19 = holiday(2019)
for i,h in enumerate(holidays):
    mergedata[h]=False
    mergedata.loc[mergedata.date==holiday18[i],h]=True
    mergedata.loc[mergedata.date==holiday19[i],h]=True
    

In [276]:
mergedata.drop(columns=['week','date'],inplace=True)